In [2]:
import ttkbootstrap as ttk
from ttkbootstrap.constants import *

    
root = ttk.Window(title="YT_StreamGrabber",
                  size=(573, 185),
                  resizable=(False, False))

root.attributes('-topmost',True)
root.place_window_center()


bg_color = "#ece7f2"

lbl_url = ttk.Label(root, text="YT-URL:") #, bg=bg_color)
lbl_url.pack(side=LEFT, padx=5, pady=10)

txt_folder = ttk.Entry(width=60)
txt_folder.pack(side=LEFT, padx=5, pady=10)

b1 = ttk.Button(root, text="Paste from Clipboard...", bootstyle=SUCCESS)
b1.pack(side=LEFT, padx=5, pady=10)

# b2 = ttk.Button(root, text="Button 2", bootstyle=(INFO, OUTLINE))
# b2.pack(side=LEFT, padx=5, pady=10)


root.mainloop()


### Necessary imports

In [ ]:
import re
from enum import Enum
from pathlib import Path
from tqdm.auto import tqdm
import webbrowser
from pytube import YouTube, Playlist

### Functions

In [ ]:
class StreamType(Enum):
    Audio = 1
    Video = 2

def download_yt_stream(url=None, dest_folder=None, stream_type=StreamType.Video, open_download_folder=False):     
    if url is None or len(url.strip()) == 0: 
        url = input()
    
    dest_folder = dest_folder if dest_folder is not None and dest_folder != '' and Path(dest_folder).exists() else ''
               
    try:
        yt = YouTube(url)        
        filename = ''    
        stream = None
        if stream_type == StreamType.Audio:
            stream = yt.streams.get_audio_only()
            filename = re.sub(r'\s{2,}', ' ', stream.default_filename.replace(".mp4", ".mp3"))
        elif stream_type == StreamType.Video:
            stream = yt.streams.get_highest_resolution()
            filename = re.sub(r'\s{2,}', ' ', stream.default_filename)   
            
        stream.download(filename=Path(dest_folder, filename))      
        if open_download_folder: 
            webbrowser.open(dest_folder)
        return f'✔️ {filename}'    
    except Exception as e:
        return f'❌ {url} ► cannot be downloaded --> Exception: {e}'

def download_yt_streams(list_of_urls=None, dest_folder=None, stream_type=StreamType.Video):
    if list_of_urls is not None and len(list_of_urls) > 0:
        for url in tqdm(list_of_urls):
            print(download_yt_stream(url, dest_folder=dest_folder, stream_type=stream_type))     
        
def download_yt_streams_from_playlist(playlist_url=None, dest_folder=None, stream_type=StreamType.Video):
    if playlist_url is None or len(playlist_url.strip()) == 0: 
        playlist_url = input()   
    download_yt_streams(Playlist(playlist_url).video_urls, dest_folder=dest_folder, stream_type=stream_type)

### Download a single audio...

In [ ]:
download_yt_stream(stream_type=StreamType.Audio, open_download_folder=True)

### ... or a single video

In [ ]:
download_yt_stream()

### ... multiple videos (or audios)

In [ ]:
urls = []
while True:
    user_input = input("Enter url: ")    
    if user_input == "quit":
        break
    else: 
        urls.append(user_input.replace('Enter url: ', ''))
        
download_yt_streams(urls, stream_type=StreamType.Video)

### ... or an entire playlist

In [ ]:
download_yt_streams_from_playlist()